In [15]:
!pip install pulp
!pip install prettytable

In [16]:
import pulp as p
from prettytable import PrettyTable

In [17]:
divisions = [1, 2, 3, 4, 5]  # Thanthirimale, Pulmude, Rambewa, Tirappane, Rajanganaya
crops = ['G', 'S', 'O']      # G = Black gram, S = Sesame, O = Big onion
division_name = {
    1: 'Thanthirimale',
    2: 'Pulmude',
    3: 'Rambewa',
    4: 'Tirappane',
    5: 'Rajanganaya'
}
crop_name = {
    'G': 'Black gram',
    'S': 'Sesame',
    'O': 'Big onion'
}

In [7]:
# Land limits (in acres) per division
land_limit = {
    1: 2000,
    2: 2300,
    3: 600,
    4: 1100,
    5: 500
}

# Water limits (in acre-feet) per division (initial)
water_limit_total = 7400  # Sum of all division limits

# Crop water requirement per acre
water_req = {
    'G': 1.6,
    'S': 2.9,
    'O': 3.5
}

# Profit per acre
profit = {
    'G': 100000,
    'S': 60000,
    'O': 110000
}

# Yield per acre
yield_per_acre = {
    'G': 50,      # bushels
    'S': 1.5,     # tons
    'O': 2.2      # tons
}

# Market constraints (total yield)
market_limit = {
    'G': 110000,   # bushels
    'S': 1800,     # tons
    'O': 2200      # tons
}

# Minimum sesame requirement (for export)
min_sesame = 800


In [9]:
#Using linear programming model to solve

model = p.LpProblem("Crop_Land_Optimization", p.LpMaximize)
x = p.LpVariable.dicts("x", ((i, j) for i in divisions for j in crops), lowBound=0, cat='Continuous')

model += (
    p.lpSum(profit[j] * x[i, j] for i in divisions for j in crops)
), "Total_Profit"

for i in divisions:
    model += p.lpSum(x[i, j] for j in crops) <= land_limit[i], f"Land_limit_div_{i}"
model += (
    p.lpSum(water_req[j] * x[i, j] for i in divisions for j in crops) <= water_limit_total
), "Total_Water_Constraint"

model += p.lpSum(yield_per_acre['G'] * x[i, 'G'] for i in divisions) <= market_limit['G'], "Market_G"
model += p.lpSum(yield_per_acre['S'] * x[i, 'S'] for i in divisions) <= market_limit['S'], "Market_S"
model += p.lpSum(yield_per_acre['O'] * x[i, 'O'] for i in divisions) <= market_limit['O'], "Market_O"

model += p.lpSum(yield_per_acre['S'] * x[i, 'S'] for i in divisions) >= min_sesame, "Min_Sesame"

model.solve()




1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
print(f"Status: {p.LpStatus[model.status]}")
print(f"Maximum Profit: Rs {p.value(model.objective):,.2f}\n")

# Create a PrettyTable
table = PrettyTable()
table.field_names = ["Division", "Crop", "Acres Allocated"]

# Fill the table
for i in divisions:
    for j in crops:
        acres = x[i, j].varValue
        if acres is not None:
            table.add_row([division_name[i], crop_name[j], f"{acres:.2f}"])

# Print the table
print(table)

Status: Optimal
Maximum Profit: Rs 325,333,330.20

+---------------+------------+-----------------+
|    Division   |    Crop    | Acres Allocated |
+---------------+------------+-----------------+
| Thanthirimale | Black gram |     1033.33     |
| Thanthirimale |   Sesame   |       0.00      |
| Thanthirimale | Big onion  |      166.67     |
|    Pulmude    | Black gram |       0.00      |
|    Pulmude    |   Sesame   |       0.00      |
|    Pulmude    | Big onion  |       0.00      |
|    Rambewa    | Black gram |      66.67      |
|    Rambewa    |   Sesame   |      533.33     |
|    Rambewa    | Big onion  |       0.00      |
|   Tirappane   | Black gram |     1100.00     |
|   Tirappane   |   Sesame   |       0.00      |
|   Tirappane   | Big onion  |       0.00      |
|  Rajanganaya  | Black gram |       0.00      |
|  Rajanganaya  |   Sesame   |       0.00      |
|  Rajanganaya  | Big onion  |      500.00     |
+---------------+------------+-----------------+
